# Network Testing

## Includes

In [ ]:
# mass includes
import os, sys
import ipdb
import pickle
import torch as t
import torchvision as tv
from PIL import Image

# add paths for all sub-folders
paths = [root for root, dirs, files in os.walk("./")]
for item in paths:
    sys.path.append(item)

from ipynb.fs.full.config import Config
from ipynb.fs.full.network import MobileNetV2
from ipynb.fs.full.util import *

## Testing

In [ ]:
# for debugging only
%pdb off

# load data
model_path = './saves/MobileNetV2_0114-230953.pth'
input_path = './doge.jpg'

# imageNet mean and std
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# class labels
with open('./products/labels.pkl', 'rb') as pkl:
    labels = pickle.load(pkl)

# choose GPU if available
device = t.device('cuda' if t.cuda.is_available() else 'cpu')

# load pre-trained model
opt = Config()
model = MobileNetV2(train=False).to(device)
model.load_state_dict(t.load(model_path, map_location=device))
model.eval()

# run network once
transforms = tv.transforms.Compose([
    tv.transforms.CenterCrop(opt.img_size),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
input_img = Image.open(input_path)
input_img = transforms(input_img).to(device)
with t.no_grad():
    result = model(input_img.unsqueeze(0))
    prob, index = t.max(result, 1)
print([labels[index.item()], prob.item()])

# export to onnx model and run intel optimizer
save_path = os.path.join('./products', '%s.onnx' % model.model_name)
t.onnx.export(
    model,
    input_img.unsqueeze(0),
    save_path,
    verbose=False,
    input_names=['input_img'],
    output_names=['result'])
out_status = os.system(
    'python3 /opt/intel/computer_vision_sdk/deployment_tools/model_optimizer/mo.py'
    + ' --input_model ./products/%s.onnx --output_dir ./products' %
    model.model_name + ' --mean_values input_img[%.3f,%.3f,%.3f]' %
    (IMAGENET_MEAN[0] * 255, IMAGENET_MEAN[1] * 255, IMAGENET_MEAN[2] * 255) +
    ' --scale_values input_img[%.3f,%.3f,%.3f]' %
    (IMAGENET_STD[0] * 255, IMAGENET_STD[1] * 255, IMAGENET_STD[2] * 255))
if out_status == 0:
    print('Model conversion completed.')